<a href="https://colab.research.google.com/github/muhardianab/web-app-stock-market-prediction/blob/main/Prophet_Stock_Market_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit

     |████████████████████████████████| 7.9 MB 34.3 MB/s 
     |████████████████████████████████| 75 kB 3.9 MB/s 
     |████████████████████████████████| 170 kB 60.5 MB/s 
     |████████████████████████████████| 4.2 MB 49.3 MB/s 
     |████████████████████████████████| 111 kB 61.3 MB/s 
     |████████████████████████████████| 122 kB 48.4 MB/s 
     |████████████████████████████████| 786 kB 42.2 MB/s 
     |████████████████████████████████| 368 kB 59.4 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13479 sha256=4ed7f51f2db3a6aab4719f98abe74a833b3eda51cc3f1035af934c5f1abe646b
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built blinker
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attemptin

In [ ]:
!pip install fbprophet

In [ ]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 40.9 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=8feef8d8e0a550a4870defbb6e984a0351072d7c4620831dad454f8eca2cdc22
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2021-07-26 07:27:34--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.195.15.150, 34.195.88.198, 3.218.70.191, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.195.15.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  53.5MB/s    in 0.2s    

2021-07-26 07:27:34 (53.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]



In [ ]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
!cp /drive/ngrok-ssh/ngrok /ngrok
/content/ngrok

In [ ]:
!/content/ngrok authtoken 1vZG7AvH8toEFJqlOS1qxSFPTmM_7MCZZVgnBriQQ7ftBqJ9r

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
%%writefile stock-market-prediction.py

import streamlit as st
from datetime import date
import yfinance as yf
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
from plotly import graph_objs
import pandas as pd

start_date = "2016-01-01"   # check first date each Stocks went to IPO
today = date.today().strftime("%Y-%m-%d")

st.title("Stock Prediction App")

stocks = ('ANTM.JK', 'MDKA.JK', 'DKFT.JK', 'INCO.JK', 'PSAB.JK', 'TINS.JK')
selected_stock = st.selectbox("Select dataset for prediction", stocks)

n_years = st.slider("Years of prediction:", 1, 2)
period = n_years * 365

@st.cache
def load_data(ticker):
    data = yf.download(ticker, start_date, today)
    data.reset_index(inplace=True)
    return data

data_load_state = st.text("Load data...")
data = load_data(selected_stock)
data_load_state.text("Loading data... done!")

st.subheader("Raw data")
st.write(data.head(3))
st.write(data.tail(5))

# Plot raw data
def plot_raw_data():
	fig = graph_objs.Figure()
	fig.add_trace(graph_objs.Scatter(x=data['Date'], y=data['Close'], name="stock_close"))
	fig.layout.update(title_text='Time Series data with Range Slider', xaxis_rangeslider_visible=True)
	st.plotly_chart(fig)
 
plot_raw_data()

# Predict with Prophet.
df_train = data[['Date','Close']]
df_train = df_train.rename(columns={"Date": "ds", "Close": "y"})

# load saved model
#with open('forecast.pckl', 'rb') as fin:
#    model = pickle.load(fin)

st.text("Wait for the data to train..")

model = Prophet()
model.fit(df_train)

st.text("Data trained done!")

future = model.make_future_dataframe(periods=period)
forecast = model.predict(future)

# Show and plot forecastion
st.subheader('Forecast data')
st.write(forecast.tail())

st.write(f'Prediction plot for {n_years} years')
fig_1 = plot_plotly(model, forecast)
st.plotly_chart(fig_1)

st.write("Prediction components")
fig_2 = model.plot_components(forecast)
st.write(fig_2)

Writing stock-market-prediction.py


In [ ]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
!streamlit run stock-market-prediction.py&>/dev/null&

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and the go to the following URL: http://1924e9d0fe48.ngrok.io
